# Fetch IRSA Dust E(B-V)

When running BADASS for multiple spectra using the multiprocessing notebook, BADASS will hangup when trying to fit spectra it has not previously obtained E(B-V) values for via Astroquery's [IRSA Dust Extinction Service Query](https://astroquery.readthedocs.io/en/latest/irsa/irsa_dust.html).  This is a known issue (see [this](https://github.com/astropy/astroquery/issues/684).  The problem stems from the fact that `IrsaDust. get_query_table()` treats multiple Python subprocesses as a single-process. For example, if you are running 4 subprocesses (fitting 4 spectra simultaneously), it will only query the last process of the four, and leave the first three hanging.  

Luckily there is a workaround.  `IrsaDust. get_query_table()` stores previous queries on your local machine so they can be accessed without looking them up every single time.  The solution is to simply query E(B-V) values for all of your objects before fitting, which seems dumb but it's the only workaround and its quick.   

In [1]:
import numpy as np
from astropy.io import fits
from astroquery.irsa_dust import IrsaDust
import astropy.units as u
from astropy import coordinates
import glob
import natsort
import sys

In [2]:
def fetch_IRSA_dust(spec_fold):
    # Get path of of spectra file
    spec_file = glob.glob(spec_fold+'/*.fits')[0]
    # Get object name 
    obj_name = spec_fold.split('/')[-1]
    
    # Load the data
    hdu = fits.open(spec_file)
    header_cols = [i.keyword for i in hdu[0].header.cards]
    if ("RA" in header_cols) and ("DEC" in header_cols):
        ra  = hdu[0].header['RA']
        dec = hdu[0].header['DEC']

        co = coordinates.SkyCoord(ra=ra, dec=dec, unit=(u.deg, u.deg), frame='fk5')
        table = IrsaDust.get_query_table(co,section='ebv')
#         print(IrsaDust.cache_location)
        ebv = table['ext SandF mean'][0]

        hdu.close()
        print(' %s,  %0.3f' % (obj_name, ebv))
        return ebv
    else: 
        return None

### Directories


In [3]:
spec_dir  = '../examples/'

spec_fold = natsort.natsorted(glob.glob(spec_dir+'*'))
print(len(spec_fold))
spec_fold = spec_fold[:20]

22


### Run

In [4]:
for i in range(len(spec_fold)):
    ebv = fetch_IRSA_dust(spec_fold[i])
    
print('\n Done')

 0-test,  0.017
 1-test,  0.041
 2-test,  0.032
 3-test,  0.088
 4-test,  0.039
 5-test,  0.023
 6-test,  0.011
 7-test,  0.019
 8-test,  0.022
 9-test,  0.006
 10-test,  0.011
 11-test,  0.035
 12-test,  0.106
 13-test,  0.064
 14-test,  0.093
 15-test,  0.036
 16-test,  0.008
 18-test,  0.023
 19-test,  0.052

 Done
